Este notebook proporciona un ejemplo práctico de cómo utilizar el modelo RoBERTa pre-entrenado para analizar y clasificar las emociones presentes en las letras de canciones. Utilizamos el paquete `transformers` de Hugging Face para cargar el modelo y el tokenizer, y luego aplicamos estos a un conjunto de datos de letras de canciones para extraer y calcular las probabilidades de diversas emociones.

## Pasos Principales

1. **Importación de Bibliotecas**: Importamos las bibliotecas necesarias, incluyendo `pandas` para el manejo de datos, `torch` para el cálculo de tensores, y `transformers` para cargar el modelo y el tokenizer.

2. **Carga del Modelo y Tokenizer**: Utilizamos el modelo `AutoModelForSequenceClassification` y el tokenizer `AutoTokenizer` desde el pre-entrenamiento de "SamLowe/roberta-base-go_emotions", que está diseñado para clasificar texto en varias emociones.

3. **Preparación de Datos**: Leemos un conjunto de datos de letras de canciones desde un archivo CSV y seleccionamos las primeras  500 palabras de cada canción para el análisis.

4. **Clasificación de Emociones**: Definimos una función para clasificar las emociones en un texto dado, utilizando el modelo y el tokenizer cargados. Calculamos las probabilidades de cada emoción y normalizamos estas probabilidades para obtener un conjunto de emociones de interés.

5. **Análisis de Emociones por Canción**: Iteramos sobre cada letra de canción en el conjunto de datos, clasificamos las emociones y almacenamos las probabilidades de cada emoción en el DataFrame original.

6. **Guardar Resultados**: Finalmente, guardamos el DataFrame modificado, que ahora incluye las probabilidades de las emociones de interés para cada canción, en un nuevo archivo CSV.

## Consideraciones

- Este notebook es un ejemplo básico y puede requerir ajustes y optimizaciones para trabajar con conjuntos de datos más grandes o para mejorar la precisión del análisis de emociones.
- La selección de emociones de interés y su definición depende del contexto específico de la aplicación. En este ejemplo, se han excluido algunas emociones que se consideran menos útiles para el análisis.
- La precisión del análisis de emociones puede variar dependiendo de la calidad y variedad del texto de las canciones, así como de la precisión del modelo pre-entrenado utilizado.

## Requisitos

- Python  3.6+
- Bibliotecas: `pandas`, `torch`, `transformers`
- Archivo de datos: `datasets/songs_cleaned.csv` (necesario para el análisis)

In [9]:
# Importar las bibliotecas necesarias
import warnings 
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification


# Ignorar las advertencias de deprecación
def fxn():
    warnings.warn("deprecated", DeprecationWarning)
with warnings.catch_warnings(action="ignore"):
    fxn()

# Cargar el modelo y el tokenizer
tokenizer = AutoTokenizer.from_pretrained("SamLowe/roberta-base-go_emotions")
model = AutoModelForSequenceClassification.from_pretrained("SamLowe/roberta-base-go_emotions")

# Leer el DataFrame con las letras de las canciones (solo las primeras 500 palabras)
df = pd.read_csv('datasets/songs_cleaned.csv', nrows=500)  # Asegúrate de tener el archivo

# Cargar el dataset de canciones
df = pd.read_csv('datasets\songs_cleaned.csv')  # Asegúrate de tener el archivo songs_cleaned.csv

# Elegir una fila del DataFrame (por ejemplo, la primera fila)
df_row = df.iloc[0]

# Clasificar las emociones y devolver las probabilidades
def classify_and_return_probabilities(text):
    inputs = tokenizer(text, max_length=500, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)
    return probabilities

# Texto de la canción
lyrics = df_row['Lyrics']

# Obtener las probabilidades de las emociones
probabilities = classify_and_return_probabilities(lyrics)

# Emociones que deseas calcular
emotions = {'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 
            'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'}
non_useful_emotions = {'disgust','disappointment','grief','disapproval', 'amusement', 'excitement', 'optimism','caring', 'pride', 'admiration', 'relief', 'approval', 'realization', 'surprise', 'curiosity', 'remorse', 'embarrassment', 'neutral'}
emotions_of_interest = emotions - non_useful_emotions

# Calcular la probabilidad total solo para las emociones de interés
total_probability = 0
for emotion in emotions_of_interest:
    index = model.config.label2id[emotion]
    total_probability += probabilities[0][index].item()

# Normalizar las probabilidades solo para las emociones de interés
normalized_probabilities = []
for emotion in emotions_of_interest:
    index = model.config.label2id[emotion]
    probability = probabilities[0][index].item() / total_probability
    normalized_probabilities.append((emotion.capitalize(), probability))

# Imprimir las probabilidades normalizadas
for emotion, probability in normalized_probabilities:
    print(f"{emotion.capitalize()}: {probability:.5f}")

<>:22: SyntaxWarning: invalid escape sequence '\s'
<>:22: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Ana\AppData\Local\Temp\ipykernel_2492\1809785111.py:22: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv('datasets\songs_cleaned.csv')  # Asegúrate de tener el archivo songs_cleaned.csv


Nervousness: 0.00553
Annoyance: 0.34545
Confusion: 0.01948
Fear: 0.00614
Anger: 0.05678
Love: 0.03838
Gratitude: 0.01029
Sadness: 0.03172
Desire: 0.01960
Joy: 0.46663


In [8]:
# Crear nuevas columnas para cada emoción útil
for emotion in emotions:
    df[emotion] = 0.0

# Clasificar cada letra de canción y llenar las columnas correspondientes
for i, lyrics in enumerate(df['Lyrics']):
    probabilities = classify_and_return_probabilities(lyrics)
    for j, emotion in enumerate(emotions):
        emotion_id = model.config.label2id[emotion]
        probability = probabilities[0][emotion_id].item()
        df.at[i, emotion] = probability

# Guardar el DataFrame modificado en un nuevo archivo CSV
df.to_csv('songs_with_useful_emotions_500_words.csv', index=False)

Para utilizar nuestro modelo el enfoque es similiar pero cargando desde el modelo pre-entrenado en local.